# Import packages and data

In [306]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from category_encoders import OrdinalEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

In [2]:
POLICY_DATA = '../data/datastorm_policy_data.csv'
AGENT_DATA = '../data/datastorm_agent_data.csv'

In [85]:
df = pd.read_csv(POLICY_DATA, parse_dates=['next_due_dt', 'termination_dt', 
                                           'main_holder_dob', 'spouse_dob', 
                                           'child1_dob', 'child2_dob', 
                                           'child3_dob', 'child4_dob', 
                                           'child5_dob', 
                                           'run_date' ])

c:\users\nisal_105136\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
df = df.drop(columns=['main_holder_occupation_cd', 'product_code'])

In [87]:
df.head()

,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,...,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_name,policy_code
0,20,M,INFORCE,2011/08/21,2019-02-21,NaT,NaN,M,1983-10-01,28,...,NaT,NaN,CASH,NaN,20190131,2020-07-01,C108223,AG108847,HEALTH,POL28535
1,15,M,INFORCE,2006/07/14,2019-09-14,NaT,NaN,F,1958-01-01,48,...,NaT,NaN,CASH,BUSINESSMAN,20190901,2020-07-01,C103349,AG103677,RETIREMENT,POL27771
2,20,Y,INFORCE,2018/12/28,2019-12-28,NaT,NaN,M,1967-11-01,51,...,NaT,NaN,CASH,RECYCLE RACERS,20191101,2020-07-01,C120388,AG103249,INVESTMENT,POL29786
3,15,Q,INFORCE,2018/11/06,2020-02-06,NaT,NaN,M,1989-10-01,29,...,NaT,NaN,CHEQUE,RECEPTIONIST,20191201,2020-07-01,C113155,AG106733,HEALTH,POL24635
4,15,M,LAPSED,2012/04/28,2015-06-28,NaT,OTHERS,M,1988-05-01,24,...,NaT,NaN,CASH,BUSINESSMAN,20190228,2020-07-01,C106155,AG106907,HEALTH,POL7470


# Create Labels

In [230]:
def create_labels(df, train_snap_date, label_last_date):
    snap_df = df[(df.policy_snapshot_as_on == train_snap_date) 
                    & (df.policy_status == 'INFORCE')][['client_code', 
                                                        'policy_code', 
                                                        'product_name']]
    cust_df = snap_df[['client_code']].drop_duplicates().set_index('client_code')

    label_df = df[(df.policy_snapshot_as_on > train_snap_date) 
                     & (df.policy_snapshot_as_on <= label_last_date) 
                     & (df.policy_status == 'INFORCE')][['client_code', 
                                                         'product_name', 
                                                         'policy_code', 
                                                         'policy_snapshot_as_on']]
    
    join_df = label_df.merge(snap_df[['policy_code', 'client_code']], on=['policy_code', 'client_code'], how='left', indicator=True)
    join_df['is_prev'] = join_df._merge.map({'left_only':0, 'both':1}).astype(int)
    
    #customers who can be c
    cross_sell_eligible = join_df.groupby(['client_code', 'policy_snapshot_as_on']).is_prev.sum()
    
    cross_sell_eligible.name = 'cross_sell_eligible'
    cross_sell_eligible = cross_sell_eligible[cross_sell_eligible > 0].reset_index()
    
    joined_cs_eligible = join_df.merge(cross_sell_eligible, on=['client_code', 'policy_snapshot_as_on'], how='left').dropna()
    
    labels = joined_cs_eligible[joined_cs_eligible.is_prev == 0][['client_code', 'product_name']].drop_duplicates()
    labels['value'] = 1
    pivot_labels = labels.pivot_table(values='value', index='client_code', aggfunc='sum', columns='product_name').fillna(0).astype(int)

    return cust_df.merge(pivot_labels, on='client_code', how='left').fillna(0)


# Create Client Dataset

In [212]:
def client_dataset(df, snap_date):
    df_snap = df[(df.policy_snapshot_as_on == snap_date) & (df.policy_status == 'INFORCE')]
    
    customer_specific_features = ['main_holder_gender', 'main_holder_dob',
                              'main_holder_smoker_flag', 'spouse_gender',
                               'spouse_dob', 'spouse_smoker_flag',
                               'child1_dob', 'child1_gender', 'child2_dob',
                               'child2_gender', 'child3_dob', 'child3_gender', 'child4_dob',
                               'child4_gender', 'child5_dob', 'child5_gender',
                               'main_holder_occupation', 
                               'client_code']
    
    df_cust = df_snap[customer_specific_features]

    dob_feat = ['main_holder_dob', 'spouse_dob', 'child1_dob', 'child2_dob',
           'child3_dob', 'child4_dob', 'child5_dob']

    age_feat = []
    for col in dob_feat:
        new_col = col[:-4] + '_age'
        df_cust[new_col] = pd.to_datetime(snap_date, format='%Y%m%d').year - df_cust[col].dt.year
        df_cust = df_cust.drop(columns=col)
        age_feat.append(new_col)

    df_cust_numeric = df_cust.groupby('client_code')[age_feat].median()

    df_cust_cat = df_cust.select_dtypes('object').groupby('client_code').first()

    df_cust_fin = df_cust_cat.join(df_cust_numeric)
    
    return df_cust_fin

In [191]:
snap_date = 20181231

In [203]:
df_snap = df[(df.policy_snapshot_as_on == snap_date) & (df.policy_status == 'INFORCE')]

In [204]:
customer_specific_features = ['main_holder_gender', 'main_holder_dob',
                              'main_holder_smoker_flag', 'spouse_gender',
                               'spouse_dob', 'spouse_smoker_flag',
                               'child1_dob', 'child1_gender', 'child2_dob',
                               'child2_gender', 'child3_dob', 'child3_gender', 'child4_dob',
                               'child4_gender', 'child5_dob', 'child5_gender',
                               'main_holder_occupation', 
                               'client_code']

In [210]:
df_cust = df_snap[customer_specific_features]

dob_feat = ['main_holder_dob', 'spouse_dob', 'child1_dob', 'child2_dob',
       'child3_dob', 'child4_dob', 'child5_dob']

age_feat = []
for col in dob_feat:
    new_col = col[:-4] + '_age'
    df_cust[new_col] = pd.to_datetime(snap_date, format='%Y%m%d').year - df_cust[col].dt.year
    df_cust = df_cust.drop(columns=col)
    age_feat.append(new_col)

df_cust_numeric = df_cust.groupby('client_code')[age_feat].median()

df_cust_cat = df_cust.select_dtypes('object').groupby('client_code').first()

df_cust_fin = df_cust_cat.join(df_cust_numeric)

In [211]:
df_cust_fin

,main_holder_gender,main_holder_smoker_flag,spouse_gender,spouse_smoker_flag,child1_gender,child2_gender,child3_gender,child4_gender,child5_gender,main_holder_occupation,main_holder_age,spouse_age,child1_age,child2_age,child3_age,child4_age,child5_age
client_code,,,,,,,,,,,,,,,,,
C100000,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLOUR & GRAIN MILL W,27.0,NaN,NaN,NaN,NaN,NaN,NaN
C100001,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHENA CULTIVATOR,37.0,NaN,NaN,NaN,NaN,NaN,NaN
C100002,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLOUR & GRAIN MILL W,24.0,NaN,NaN,NaN,NaN,NaN,NaN
C100003,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MINISTER,27.0,NaN,NaN,NaN,NaN,NaN,NaN
C100004,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MINISTER,46.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C154407,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SALESMAN,39.0,NaN,NaN,NaN,NaN,NaN,NaN
C154416,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SALESMAN,30.0,NaN,NaN,NaN,NaN,NaN,NaN
C154436,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN


# Fill NA values

In [228]:
def fillna_vals(df_cust):
    cat_cols = ['main_holder_gender', 'main_holder_smoker_flag', 'spouse_gender',
       'spouse_smoker_flag', 'child1_gender', 'child2_gender', 'child3_gender',
       'child4_gender', 'child5_gender', 'main_holder_occupation']

    df_cust[cat_cols] = df_cust[cat_cols].fillna('unk')

    zero_fill_cols = ['spouse_age', 'child1_age', 'child2_age',
           'child3_age', 'child4_age', 'child5_age']
    df_cust[zero_fill_cols] = df_cust[zero_fill_cols].fillna(0)
    
    return df_cust

# Categorical Encoding

In [282]:
def cat_encoding(df_cust):
    cat_cols = ['main_holder_gender', 'main_holder_smoker_flag', 'spouse_gender',
           'spouse_smoker_flag', 'child1_gender', 'child2_gender', 'child3_gender',
           'child4_gender', 'child5_gender', 'main_holder_occupation']

    cat_enc = OrdinalEncoder(cols=cat_cols, verbose=False)

    cat_enc.fit(df_cust)

    df_cust = cat_enc.transform(df_cust)
    
    return df_cust

# Create Train Dataset

In [289]:
def create_train_set(df, snap_date, last_label_date):
    df_labels = create_labels(df, snap_date, last_label_date)

    df_cust = client_dataset(df, snap_date)

    df_cust = fillna_vals(df_cust)
    
    df_cust = cat_encoding(df_cust)

    df_cross_sell = (df_labels.sum(axis=1) > 0).astype(int)
    df_cross_sell.name = 'is_cross_sell'

    return df_cust.join(df_cross_sell)

In [284]:
snap_date = 20181231
last_label_date = 20190630

In [285]:
df_labels = create_labels(df, snap_date, last_label_date)

df_cust = client_dataset(df, snap_date)

df_cust = fillna_vals(df_cust)

df_cross_sell = (df_labels.sum(axis=1) > 0).astype(int)
df_cross_sell.name = 'is_cross_sell'

df_train_test = df_cust.join(df_cross_sell)

c:\users\nisal_105136\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Model Training

In [290]:
policy_snaps = sorted(df.policy_snapshot_as_on.unique())

In [291]:
policy_snaps[19]

20200701

In [292]:
df_train = create_train_set(df, policy_snaps[0], policy_snaps[6])

c:\users\nisal_105136\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [293]:
df_test = create_train_set(df, policy_snaps[7], policy_snaps[13])

c:\users\nisal_105136\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [298]:
train_X = df_train.drop(columns='is_cross_sell')
train_y = df_train['is_cross_sell']

test_X = df_train.drop(columns='is_cross_sell')
test_y = df_train['is_cross_sell']

In [303]:
model = RandomForestClassifier(n_jobs=-1, n_estimators=500, max_depth=8, class_weight='balanced')
model.fit(train_X, train_y)

pred_y = model.predict(test_X)
pred_y_prob = model.predict_proba(test_X)[:,1]

In [307]:
print(confusion_matrix(test_y, pred_y))

[[10310  8436]
 [  128   397]]


In [304]:
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.99      0.55      0.71     18746
           1       0.04      0.76      0.08       525

    accuracy                           0.56     19271
   macro avg       0.52      0.65      0.40     19271
weighted avg       0.96      0.56      0.69     19271



In [305]:
print(roc_auc_score(test_y, pred_y_prob))

0.7217767853967576


# Model Explanations

In [312]:
pd.Series(model.feature_importances_, index=train_X.columns).sort_values(ascending=False)

main_holder_age            0.303804
main_holder_occupation     0.230457
spouse_age                 0.126310
child1_age                 0.093470
child2_age                 0.064616
main_holder_smoker_flag    0.031861
child2_gender              0.028803
child1_gender              0.025819
child3_age                 0.022201
spouse_gender              0.021587
main_holder_gender         0.020916
child3_gender              0.015151
spouse_smoker_flag         0.014556
child4_age                 0.000227
child4_gender              0.000215
child5_age                 0.000003
child5_gender              0.000003
dtype: float64